## Ejercicio Nro. 4 - Web scraping Poperati

In [1]:
# Scraping de ranking de 250 peliculas,armar tabla con algunas variables de importancia, como titulo, rating, año, etc

## Librerías

In [8]:
# Instalación
packages <- c("rvest", "data.table", "stringr", "robotstxt", "lubridate", "purrr")
if (length(setdiff(packages, rownames(installed.packages()))) > 0) {
  install.packages(setdiff(packages, rownames(installed.packages())))  
}

library("rvest")
library("data.table")
library("stringr")
library("robotstxt")
library("lubridate")
library("purrr")

## Se verifica que se pueda descargar información de la página

In [3]:
paths_allowed(
  paths = c("https://www.properati.com.ar/")
)

 www.properati.com.ar                      



[1] TRUE

## Variables de interés

In [4]:
# Información que se va a descargar de las páginas
tags <- c("tipo", "habitaciones", "m2", "precio", "moneda", "direccion", "localidad", "contacto")

# Se define la cantidad de páginas dentro del sitio que recorro para hacer scraping
numPagina <- 20

# Se crea una lista para guardar temporalmente los datos de cada página
paginas <- list()

## Navegación por página y scraping

In [5]:
# Se recorre cada página, se descarga la información requerida y se realiza la limpieza necesaria
for (n in 1:numPagina) {
  
  # Página solicitada
  html <- paste0("https://rafaela.properati.com.ar/nf/propiedades/venta/", n, "/")
  
  # Se carga la página
  propiedades <- read_html(html)
 
  # Precio de la propiedad y moneda
  prop_precio <- propiedades %>% html_nodes(".price") %>% html_text() %>% str_replace_all("[\\s]+", "")
  
  prop_valor  <- str_replace_all(prop_precio, "[^0123456789]", "")  %>% as.numeric()
  prop_moneda <- str_replace_all(prop_precio, "[^U$S]"       , "")
  
  # Dirección
  prop_direc  <- propiedades %>% html_nodes(".item-url span") %>% html_text()
  prop_direc  <- tolower(prop_direc)
  
  # Localidad  
  prop_loc    <- propiedades %>% html_nodes( ".span7") %>% html_node(".location") %>% html_text() %>% 
    str_replace_all("[\\sm²]+", "")
  prop_loc    <- tolower(prop_loc)
  
  # Tipo de propiedad
  prop_tipo   <- propiedades %>% html_nodes(".property-type") %>% html_text() 
  prop_tipo   <- tolower(prop_tipo)
  
  # Cantidad de ambientes
  prop_amb    <- propiedades %>% html_nodes( ".span7") %>% html_node(".rooms") %>% html_text() %>% 
    str_replace_all("[\\s]+", "") %>% str_sub(1,1) %>% as.numeric()
  
  # Superficie cubierta
  prop_m2     <- propiedades %>% html_nodes( ".span7") %>% html_node(".area") %>% html_text() %>% 
    str_replace_all("[\\sm²]+", "") %>% as.numeric()
 
  # Contacto
  prop_vend   <- propiedades %>% html_nodes( ".span7") %>% html_node(" .rooms+ p") %>% html_text()
  prop_vend   <- tolower(prop_vend)

  #Se crea tabla con variables
  tabla <- data.table(prop_tipo, prop_amb, prop_m2, prop_valor, prop_moneda, prop_direc, prop_loc, prop_vend, n)
  names(tabla) <- c(tags, "pag")
  
  # Si hay celdas vacías se asigna NA
  tabla[tabla == ""] <- NA
  
  # Se guarda información en lista
  paginas[[n]] <- tabla
    
  cat(paste0("Página N° ", n, "... Cantidad de registros: ", nrow(tabla), " ..."), "\n")
}

Página N° 1... Cantidad de registros: 20 ... 
Página N° 2... Cantidad de registros: 18 ... 
Página N° 3... Cantidad de registros: 20 ... 
Página N° 4... Cantidad de registros: 18 ... 
Página N° 5... Cantidad de registros: 20 ... 
Página N° 6... Cantidad de registros: 20 ... 
Página N° 7... Cantidad de registros: 20 ... 
Página N° 8... Cantidad de registros: 20 ... 
Página N° 9... Cantidad de registros: 20 ... 
Página N° 10... Cantidad de registros: 20 ... 
Página N° 11... Cantidad de registros: 20 ... 
Página N° 12... Cantidad de registros: 20 ... 
Página N° 13... Cantidad de registros: 20 ... 
Página N° 14... Cantidad de registros: 20 ... 
Página N° 15... Cantidad de registros: 20 ... 
Página N° 16... Cantidad de registros: 20 ... 
Página N° 17... Cantidad de registros: 20 ... 
Página N° 18... Cantidad de registros: 20 ... 
Página N° 19... Cantidad de registros: 20 ... 
Página N° 20... Cantidad de registros: 20 ... 


## Se transforma la lista creada en una tabla única

In [6]:
dt_propiedades <- rbindlist(paginas)
head(dt_propiedades, 10)

tipo,habitaciones,m2,precio,moneda,direccion,localidad,contacto,pag
casa,1,NA,4650000,$,bolivar y cervantes 100,rafaela,odn + oficina de negocios inmobiliarios,1
ph,1,NA,70000,U$S,rincon 700,rafaela,odn + oficina de negocios inmobiliarios,1
casa,1,NA,130000,U$S,washington 100,rafaela,odn + oficina de negocios inmobiliarios,1
casa,1,NA,2500000,$,balbi 2100,rafaela,odn + oficina de negocios inmobiliarios,1
casa,NA,270,95000,U$S,aristobulo del valle al 1300,rafaela,NA,1
casa,NA,300,79500,U$S,larrea al 2500,rafaela,NA,1
casa,1,NA,80000,$,e. salva 700,rafaela,odn + oficina de negocios inmobiliarios,1
terreno,1,300,33000,U$S,intendente gimenez 1600,rafaela,odn + oficina de negocios inmobiliarios,1
terreno,1,1288,140000,U$S,rojas y las colonias 100,rafaela,odn + oficina de negocios inmobiliarios,1
casa,1,NA,120000,U$S,liniers 400,rafaela,odn + oficina de negocios inmobiliarios,1


In [7]:
# Se ordenan propiedades por precio
setorder(dt_propiedades, -precio, na.last = TRUE)
head(dt_propiedades, 10)

tipo,habitaciones,m2,precio,moneda,direccion,localidad,contacto,pag
casa,1,NA,4650000,$,bolivar y cervantes 100,rafaela,odn + oficina de negocios inmobiliarios,1
casa,1,NA,2500000,$,balbi 2100,rafaela,odn + oficina de negocios inmobiliarios,1
departamento,1,48,1500000,$,carlos nicola 100,rafaela,odn + oficina de negocios inmobiliarios,1
departamento,1,48,1500000,$,carlos nicola 100,rafaela,odn + oficina de negocios inmobiliarios,2
otro,NA,265,800000,$,sabin 570,rafaela,NA,2
galpón,NA,400,725000,U$S,avenida santa fe al 1400,rafaela,NA,2
otro,NA,360,400000,$,aconcagua y carrero podio,rafaela,NA,1
otro,NA,360,400000,$,aconcagua y carrero podio,rafaela,NA,2
casa,NA,336,250000,U$S,américa y trento al 100,rafaela,NA,1
casa,NA,2000,150000,U$S,calle sin nombre,rafaela,NA,2
